In [1]:
import librosa
import csv
from scipy.signal import find_peaks
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import moviepy.editor as mp

In [2]:
# Get the duration of the audio file
audio_file = "chopin.wav"
duration = librosa.get_duration(path=audio_file)

print("Duration of audio:", duration)

# Load the audio file
y, sr = librosa.load(audio_file, sr=None)

# Define chunk size and overlap
chunk_size = 8192  # Increase chunk size for higher resolution
overlap = 2048  # Ensure overlap is smaller than chunk size

# Calculate number of chunks
num_chunks = (len(y) - overlap) // (chunk_size - overlap)

# MIDI note number for A4 (440 Hz)
A4_note_number = 69

# Function to convert MIDI note number to frequency
def note_number_to_frequency(note_number):
    return 440 * 2 ** ((note_number - 69) / 12)

# Create a set to store unique note names
note_names = set()

# Loop over chunks to find all unique note names
for i in range(num_chunks):
    start = i * (chunk_size - overlap)
    end = start + chunk_size
    chunk = y[start:end]

    # Perform FFT on the chunk
    fft_result = np.fft.fft(chunk)

    # Calculate frequencies corresponding to FFT result
    frequencies = np.fft.fftfreq(len(chunk), d=1/sr)

    # Keep only positive frequencies
    positive_frequencies = frequencies[:len(chunk)//2]
    magnitude_spectrum = np.abs(fft_result)[:len(chunk)//2]

    # Limit frequencies to 4,000 Hz
    max_frequency_index = np.argmax(positive_frequencies >= 5000)
    positive_frequencies = positive_frequencies[:max_frequency_index]
    magnitude_spectrum = magnitude_spectrum[:max_frequency_index]

    # Find peaks above a certain threshold
    peaks, _ = find_peaks(magnitude_spectrum, height=10)

    # Fill set with note names for each peak
    for peak in peaks:
        frequency = positive_frequencies[peak]
        if frequency > 0:  # Check if frequency is positive
            try:
                note_number = round(12 * np.log2(frequency / note_number_to_frequency(A4_note_number)) + A4_note_number)
                if note_number >= 0 and note_number <= 127:
                    note_name = librosa.core.midi_to_note(note_number)
                    note_names.add(note_name)  # Collect note name
            except (ValueError, ZeroDivisionError, OverflowError):
                pass  # Skip frequencies resulting in errors

# Sort note names by their MIDI note numbers
sorted_note_names = sorted(list(note_names), key=lambda x: librosa.core.note_to_midi(x))

# Create a list to store data
data = []

# Loop over chunks again to process data
for i in range(num_chunks):
    start = i * (chunk_size - overlap)
    end = start + chunk_size
    chunk = y[start:end]

    # Perform FFT on the chunk
    fft_result = np.fft.fft(chunk)

    # Calculate frequencies corresponding to FFT result
    frequencies = np.fft.fftfreq(len(chunk), d=1/sr)

    # Keep only positive frequencies
    positive_frequencies = frequencies[:len(chunk)//2]
    magnitude_spectrum = np.abs(fft_result)[:len(chunk)//2]

    # Limit frequencies to 4,000 Hz
    max_frequency_index = np.argmax(positive_frequencies >= 4000)
    positive_frequencies = positive_frequencies[:max_frequency_index]
    magnitude_spectrum = magnitude_spectrum[:max_frequency_index]

    # Find peaks above a certain threshold
    peaks, _ = find_peaks(magnitude_spectrum, height=10)

    # Create a dictionary to store amplitude values for each note
    note_amplitudes = {}

    # Fill dictionary with amplitude values for peaks
    for peak in peaks:
        frequency = positive_frequencies[peak]
        if frequency > 0:  # Check if frequency is positive
            try:
                note_number = round(12 * np.log2(frequency / note_number_to_frequency(A4_note_number)) + A4_note_number)
                if note_number >= 0 and note_number <= 127:
                    note_name = librosa.core.midi_to_note(note_number)
                    amplitude_value = magnitude_spectrum[peak]
                    note_amplitudes[note_name] = amplitude_value
            except (ValueError, ZeroDivisionError, OverflowError):
                pass  # Skip frequencies resulting in errors
    
    # Map amplitude values to frame data
    frame_data = [note_amplitudes.get(note, 0) for note in sorted_note_names]

    data.append(frame_data)  # Append frame data to main data list

# Write data to CSV file
csv_file = "output.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write header row
    writer.writerow(sorted_note_names)
    
    # Write data rows
    for row in data:
        formatted_row = ['%.2f' % round(elem, 2) if elem != 0 else '0' for elem in row]
        writer.writerow(formatted_row)


# Read the CSV file
try:
    df = pd.read_csv(csv_file)

    # Drop columns with all zeros
    df = df.loc[:, (df != 0).any(axis=0)]

    # Write the modified DataFrame back to CSV
    df.to_csv("processed_output.csv", index=False)

    # Print number of rows and columns
    num_rows, num_columns = df.shape
    print(f"\nNumber of Frames: {num_rows}")
    print(f"Number of Notes Present: {num_columns}")
except pd.errors.EmptyDataError:
    print("EmptyDataError: No columns to parse from file")


Duration of audio: 25.08222222222222

Number of Frames: 179
Number of Notes Present: 68


In [4]:
# Read the CSV file
df = pd.read_csv("processed_output.csv")

# Find the highest cell value within the first 10 rows
highest_value = df.iloc[:10].max().max()

# Filter out zeros and find the lowest non-zero value
lowest_nonzero_value = df[df != 0].min().min()

# Define the frequency range corresponding to visible light (approximate)
# Here, we'll consider the range of audible frequencies
lowest_freq = 20  # Hz (lowest audible frequency)
highest_freq = 20000  # Hz (highest audible frequency)

# Set the doubled figure size
figsize = (12, 6)

# Create a directory to store frames
frames_directory = "frames"
os.makedirs(frames_directory, exist_ok=True)

# Create a figure and axis
fig, ax = plt.subplots(figsize=figsize)

# Interpolation function
def interpolate_data(row1, row2, alpha):
    return row1 * (1 - alpha) + row2 * alpha

# Function to update the plot for each frame
def update(frame):
    ax.clear()
    row1 = df.iloc[frame]
    if frame + 1 < len(df):
        row2 = df.iloc[frame + 1]
        for alpha in np.linspace(0, 1, 10):  # Interpolate 10 frames between each pair of rows
            interpolated_row = interpolate_data(row1, row2, alpha)
            plot_row(interpolated_row)
    else:
        plot_row(row1)
    
    ax.set_aspect('equal')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_title(f'Frame {frame+1}')

# Function to plot circles for a given row
def plot_row(row):
    nonzero_values = row[row != 0]
    for value in nonzero_values:
        radius = (value / (highest_value - lowest_nonzero_value)) * 0.5
        frequency = lowest_freq + (value - lowest_nonzero_value) / (highest_value - lowest_nonzero_value) * (highest_freq - lowest_freq)
        color = plt.cm.plasma(frequency / highest_freq)
        circle = plt.Circle((0.5, 0.5), radius, color=color, alpha=0.5)
        ax.add_artist(circle)

# Function to save each frame
def save_frame(frame, idx):
    ax.clear()
    update(frame)
    plt.savefig(f"{frames_directory}/frame_{idx:04d}.png", bbox_inches="tight")

# Iterate through frames and save each frame
for idx in range(len(df)):
    save_frame(idx, idx)

# Close the plot to avoid displaying it
plt.close()

# Load the audio clip
audio_clip = mp.AudioFileClip("chopin.wav")

# Create a list of frame filenames
frame_filenames = [f"{frames_directory}/frame_{idx:04d}.png" for idx in range(len(df))]

# Create a VideoClip from the frames with a lower fps and longer duration
video_clip = mp.ImageSequenceClip(frame_filenames, fps=10)  # Decrease fps to slow down the animation
video_duration = len(df) / 10  # Adjust the duration based on the number of frames and fps
final_clip = video_clip.set_audio(audio_clip).set_duration(video_duration)

# Write the synchronized animation with audio to a file with a different audio codec
final_clip.write_videofile("output.mp4", codec="libx264", audio_codec="aac")

# Clean up: remove the frame images
for filename in frame_filenames:
    os.remove(filename)

Moviepy - Building video output.mp4.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output.mp4



Moviepy - Done !
Moviepy - video ready output.mp4


In [6]:
# Read the CSV file
df = pd.read_csv("processed_output.csv")

# Find the highest cell value within the first 10 rows
highest_value = df.iloc[:10].max().max()

# Filter out zeros and find the lowest non-zero value
lowest_nonzero_value = df[df != 0].min().min()

# Define the frequency range corresponding to visible light (approximate)
# Here, we'll consider the range of audible frequencies
lowest_freq = 60  # Hz (lowest audible frequency)
highest_freq = 7500  # Hz (highest audible frequency)

# Set the doubled figure size
figsize = (12, 6)

# Create a directory to store frames
frames_directory = "frames"
os.makedirs(frames_directory, exist_ok=True)

# Create a figure and axis
fig, ax = plt.subplots(figsize=figsize)

# Interpolation function
def interpolate_data(row1, row2, alpha):
    return row1 * (1 - alpha) + row2 * alpha

# Function to update the plot for each frame
def update(frame):
    ax.clear()
    row1 = df.iloc[frame]
    if frame + 1 < len(df):
        row2 = df.iloc[frame + 1]
        for alpha in np.linspace(0, 1, 10):  # Interpolate 10 frames between each pair of rows
            interpolated_row = interpolate_data(row1, row2, alpha)
            plot_row(interpolated_row)
    else:
        plot_row(row1)
    
    ax.set_aspect('equal')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_title(f'Frame {frame+1}')

# Function to plot circles for a given row
def plot_row(row):
    nonzero_values = row[row != 0]
    for value in nonzero_values:
        radius = (value / (highest_value - lowest_nonzero_value)) * 0.5
        frequency = lowest_freq + (value - lowest_nonzero_value) / (highest_value - lowest_nonzero_value) * (highest_freq - lowest_freq)
        color = plt.cm.plasma(frequency / highest_freq)
        circle = plt.Circle((0.5, 0.5), radius, color=color, alpha=0.5)
        ax.add_artist(circle)

# Function to save each frame
def save_frame(frame, idx):
    ax.clear()
    update(frame)
    plt.savefig(f"{frames_directory}/frame_{idx:04d}.png", bbox_inches="tight")

# Iterate through frames and save each frame
for idx in range(len(df)):
    save_frame(idx, idx)

# Close the plot to avoid displaying it
plt.close()

# Load the audio clip
audio_clip = mp.AudioFileClip("chopin.wav")

# Create a list of frame filenames
frame_filenames = [f"{frames_directory}/frame_{idx:04d}.png" for idx in range(len(df))]

# Calculate the desired duration of the animation (in seconds)
desired_duration = 20  # Adjust this value as needed

# Calculate the desired frames per second (fps) to achieve the desired duration
desired_fps = len(df) / desired_duration

# Decrease fps if the desired fps exceeds the original fps of the frames
if desired_fps > 30:  # Assuming the original fps is 30, adjust this if needed
    desired_fps = 30  # Limit to the original fps to avoid too fast animation

# Calculate the actual duration of the animation based on the desired fps
actual_duration = len(df) / desired_fps

# Create a VideoClip from the frames with the calculated fps and duration
video_clip = mp.ImageSequenceClip(frame_filenames, fps=desired_fps)
final_clip = video_clip.set_audio(audio_clip).set_duration(actual_duration)

# Write the synchronized animation with audio to a file with a different audio codec
final_clip.write_videofile("output2.mp4", codec="libx264", audio_codec="aac")

# Clean up: remove the frame images
for filename in frame_filenames:
    os.remove(filename)

Moviepy - Building video output2.mp4.
MoviePy - Writing audio in output2TEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output2.mp4



Moviepy - Done !
Moviepy - video ready output2.mp4
